# Fine-tuning an Image Classifier

### We will fine-tune and alread pre-trained image classifier in Keras

This notebook uses the script "fine-tune_classifier.py" and is meant to help *visualize* the training process. See the notebook "evaluate_classifier.ipynb" for methods on evaluating the image classifier.

The following utilizes a **"Locations.py"** (not included) script that specifies the locations of various repositories on the local machine. Thus, there needs to be slight adjustments if you have downloaded this file from Github. Specifically, you must specify the following (among potentially others):


 - **test_data_dir** = *location of test data/images*
 
 - **preview_dir** = *where to store the augmented images that are previewed (optional)*
 
 - **runs_dir** = *location of saved weights*
 
 - **arch_dir** = *location of saved model architectures*
 
 - **model_location** = *specific location of the desired model within the arch_dir*
 
 - **train_bottleneck** = *where are the training bottleneck features stored?*
 
 - **validation_bottleneck** = *where are the validating bottleneck features stored?*
 
 - **plot_dir** = *location of saved plots*

In [ ]:
%matplotlib inline

In [ ]:
#import modules/dependencies
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import os
import sys
import tarfile
import random
from IPython.display import display, Image
from scipy import ndimage
from IPython.display import SVG
from keras.models import model_from_json
from keras.utils import np_utils
from keras.utils.visualize_util import model_to_dot
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
#First, we run the script "fine-tune_classifier.py"
# or %run train_classifier.py ?
exec(open("fine-tune_classifier.py").read())

In [ ]:
# access locations file (if needed)
exec(open("Locations.py").read())

In [ ]:
#train model
fully_tuned_model = fine_tuning()

In [ ]:
#Optiona, display summary
fully_tuned_model.summary()

In [ ]:
#Optional, save weights. Note that weights auto saved when trained
save_weights(which_model = fully_tuned_model, filename = "fully_tuned_VGG16_complete")

In [ ]:
#Optional, save plot
save_plot(which_model = fully_tuned_model, plotname = "fully_tuned_VGG16")

In [ ]:
#Optional, display plot
SVG(model_to_dot(fully_tuned_model).create(prog='dot', format='svg'))

In [ ]:
#Display example cat (resized)
image_cat = "cats/cat.98 copy.jpg"
image_dog = "dogs/dog.91 copy.jpg"
image_path_cat = os.path.join(test_data_dir, image_cat)
image_path_dog = os.path.join(test_data_dir, image_dog)

a = Image(filename = image_path_cat, width = 150, height = 150)
b = Image(filename = image_path_dog, width = 150, height = 150)
display(a, b)

In [ ]:
#optional (alread ran), create example augmented images
img = load_img(new_image_path)  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

#example datagen
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=25,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        zoom_range=0.2,
        fill_mode = 'nearest',
        vertical_flip = True,
        horizontal_flip=True)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
directory = preview_dir + "/cats"
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir=directory, save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely


In [ ]:
#display augmented images (actual size)
a = Image(filename = preview_dir + "/cats" + "/cat_0_2043.jpeg", width = 150, height = 150)
b = Image(filename = preview_dir + "/cats" + "/cat_0_2058.jpeg", width = 150, height = 150)
c = Image(filename = preview_dir + "/cats" + "/cat_0_2215.jpeg", width = 150, height = 150)
d = Image(filename = preview_dir + "/cats" + "/cat_0_3130.jpeg", width = 150, height = 150)
e = Image(filename = preview_dir + "/cats" + "/cat_0_3474.jpeg", width = 150, height = 150)
display(a, b, c, d, e)

In [ ]:
#Optional, load an already trained model into notebook
def load_model(arch = model_location, which_model = "original_model_complete"):
    run = os.path.join(runs_dir, which_model)
    model = model_from_json(open(arch).read())
    model.load_weights(run)
    print("model loaded.")
    return model

model = load_model(arch_dir + "/fully_tuned_VGG16.json", runs_dir + "/fully_tuned_VGG16")

In [ ]:
#use generator to rescale and predict on test data set
rescale_datagen = ImageDataGenerator(rescale=1./255)

rescale_generator = rescale_datagen.flow_from_directory(
        directory = test_data_dir,
        target_size=(img_width, img_height),
        color_mode = "rgb",
        classes = ['cats', 'dogs'],
        batch_size=1,
        shuffle = False,
        class_mode='categorical')

a = Image(filename = image_path_cat, width = 150, height = 150)
b = Image(filename = image_path_dog, width = 150, height = 150)
display(a, b)
print(model.predict_generator(rescale_generator, 2))

In [ ]:
#load and scale images to be tested on (different way to rescale, different answer?)
img1 = test_data_dir + "/dogs/oscar1.jpg"
img2 = test_data_dir + "/dogs/oscar2.jpg"
img3 = test_data_dir + "/dogs/oscar3.jpg"
img4 = test_data_dir + "/dogs/oscar4.jpg"
img5 = test_data_dir + "/dogs/oscar5.jpg"
o1 = Image(filename = img1, width = 150, height = 150)
o2 = Image(filename = img2, width = 150, height = 150)
o3 = Image(filename = img3, width = 150, height = 150)
o4 = Image(filename = img4, width = 150, height = 150)
o5 = Image(filename = img5, width = 150, height = 150)

img_names = [img1, img2, img3, img4, img5]

def load_and_scale_imgs(image_names = img_names):
    imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread(image_name),
                            (150, 150)), (0, 1, 2)).astype('float32') for image_name in image_names]
    return np.array(imgs) / 255

#make predictions
imgs = load_and_scale_imgs()
predictions = model.predict(imgs)
display(o1, o2, o3, o4, o5)
print(predictions)

In [ ]:
#load and scale images to be tested on (different way to rescale, different answer?)
img_names = [image_path_cat, image_path_dog]

def load_and_scale_imgs(image_names = img_names):
    imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread(image_name),
                            (150, 150)), (0, 1, 2)).astype('float32') for image_name in image_names]
    return np.array(imgs) / 255

#make predictions
imgs = load_and_scale_imgs()
predictions = model.predict_classes(imgs)
print(predictions)

In [ ]:
#predict loss on bath
predictions = model.predict_on_batch(imgs)
print(predictions)